In [1]:
## Using LSTM to detect churn in the twitter dataset
## ALso using word-embedding

In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

In [4]:
import re

In [5]:
from nltk.corpus import stopwords
stopwords=stopwords.words("english")

In [6]:
from nltk.stem import PorterStemmer

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [10]:
from sklearn.preprocessing import LabelEncoder

In [11]:
from keras.preprocessing import sequence

/home/silverstar/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [12]:
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.recurrent import SimpleRNN
from keras.layers import concatenate,Flatten,Reshape

In [13]:
from keras.models import Model

In [14]:
from keras.layers import RNN
from keras.layers import Input

In [15]:
import keras.backend as K

In [16]:
mainDF=pd.read_csv("dataV2.csv")

In [17]:
brands=np.unique(mainDF['brand'])

# Doing Pre-Processing

In [18]:
mainDF.iloc()[0].text

'gone mangy cat got go  <bb>  drop cable box remote gamestop exchange game rack screws'

In [19]:
porter=PorterStemmer()
def preprocessing(a_tweet):
    cleanTweet=[]
    a_tweet=a_tweet.split(" ")
    for word in a_tweet:
        word=word.strip()
        valid=re.search(pattern=r"[^A-Za-z<>]",string=word)==None
        if(word!="" and valid==True and word not in stopwords):
            word=porter.stem(word)
            cleanTweet.append(word)
    return " ".join(cleanTweet)

In [20]:
mainDF['text']=mainDF['text'].apply(preprocessing)

mainDF=mainDF.dropna()

In [21]:
mainDF.iloc()[0].text

'gone mangi cat got go <bb> drop cabl box remot gamestop exchang game rack screw'

In [22]:
countVect=CountVectorizer()
countVect.fit_transform(mainDF['text'])

<3919x4333 sparse matrix of type '<class 'numpy.int64'>'
	with 29181 stored elements in Compressed Sparse Row format>

In [23]:
vocab=countVect.get_feature_names()

In [24]:
def convertWord(tweet):
    tweet=tweet.split(" ")
    indexTweet=[]
    for word in tweet:
        word=word.lower()
        if word in vocab:
            indexTweet.append(vocab.index(word))
    return indexTweet

In [25]:
mainDF['text']=mainDF['text'].apply(convertWord)

In [26]:
max_tweet_length=-1
max_tweet_index=0
for tweet in mainDF['text']:
    if(len(tweet)>max_tweet_length):
        max_tweet_length=len(tweet)
        max_tweet_index=tweet

In [27]:
mainDF['brand']=LabelEncoder().fit_transform(mainDF['brand'])
mainDF['label']=LabelEncoder().fit_transform(mainDF['choose_one'])

In [28]:
X=sequence.pad_sequences(mainDF['text'].ravel(),max_tweet_length)
y=mainDF['label']

In [29]:
X_train,X_test,y_train,y_test=train_test_split(X,y.ravel(),stratify=y.ravel())

In [30]:
top_words=len(vocab)
embedding_vecor_length = 32

In [ ]:
Embedding(top_words, embedding_vecor_length, input_length=max_tweet_length)

In [31]:
top_words=len(vocab)
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_tweet_length))
model.add(LSTM(3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 22, 32)            138656    
_________________________________________________________________
lstm_1 (LSTM)                (None, 3)                 432       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 4         
Total params: 139,092
Trainable params: 139,092
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30,batch_size=128,class_weight={0:6,1:1})

Train on 2939 samples, validate on 980 samples
Epoch 1/30
2939/2939 [==============================] - 0s 123us/step - loss: 0.2610 - acc: 0.9619 - val_loss: 0.6255 - val_acc: 0.7541
Epoch 2/30
2939/2939 [==============================] - 0s 109us/step - loss: 0.2450 - acc: 0.9622 - val_loss: 0.6336 - val_acc: 0.7582
Epoch 3/30
2939/2939 [==============================] - 0s 148us/step - loss: 0.2411 - acc: 0.9639 - val_loss: 0.6486 - val_acc: 0.7520
Epoch 4/30
2939/2939 [==============================] - 1s 172us/step - loss: 0.2280 - acc: 0.9629 - val_loss: 0.6543 - val_acc: 0.7551
Epoch 5/30
2939/2939 [==============================] - 0s 140us/step - loss: 0.2257 - acc: 0.9643 - val_loss: 0.6553 - val_acc: 0.7541
Epoch 6/30
2939/2939 [==============================] - 0s 114us/step - loss: 0.2199 - acc: 0.9633 - val_loss: 0.6622 - val_acc: 0.7541
Epoch 7/30
2939/2939 [==============================] - 0s 122us/step - loss: 0.2509 - acc: 0.9524 - val_loss: 0.7397 - val_acc: 0.7173
E

In [37]:
y_pred=model.predict_classes(X_test)

In [38]:
def createSummary(y_test,y_pred):
    class0List=np.where(y_test==0)
    class1List=np.where(y_test==1)
    sumDict={}
    sumDict['accuracy']=np.mean(y_test==y_pred)
    sumDict['class0Accuracy']=np.mean(y_test[class0List]==y_pred[class0List])
    sumDict['class1Accuracy']=np.mean(y_test[class1List]==y_pred[class1List])
    sumDict['precision']=precision_score(y_test,y_pred)
    sumDict['recall']=recall_score(y_test,y_pred)
    sumDict['f1-score']=f1_score(y_test,y_pred)
    sumDict['auc']=roc_auc_score(y_test,y_pred)
    sumDict['confuseMatrix']=confusion_matrix(y_test,y_pred)
    print(classification_report(y_test,y_pred,target_names=['class 0','class 1']))
#     print(sumDict)
    return sumDict

In [39]:
createSummary(y_test,y_pred)

              precision    recall  f1-score   support

     class 0       0.42      0.51      0.46       207
     class 1       0.86      0.81      0.83       773

   micro avg       0.75      0.75      0.75       980
   macro avg       0.64      0.66      0.65       980
weighted avg       0.77      0.75      0.76       980



{'accuracy': 0.6396730528946273,
 'class0Accuracy': 0.5120772946859904,
 'class1Accuracy': 0.8098318240620958,
 'precision': 0.8610729023383769,
 'recall': 0.8098318240620958,
 'f1-score': 0.8346666666666666,
 'auc': 0.6609545593740431,
 'confuseMatrix': array([[106, 101],
        [147, 626]])}

In [64]:
brands={}
brands['t-mobile']=['t-mobile','tmobile','tmobi']
brands['verizon']=['verizon','veryzon']
brands['at&t']=['att','at&t',]

def generatePrediction(tweet,brand):
    tweet=tweet.replace("&amp;","&")
    tweet=tweet.replace("amp;",'&')
    for key,value in brands.items():
        if(key==brand):
            replaceWord=" <bb> "
        else:
            replaceWord=" <aa> "
            
        for name in value:
            tweet=tweet.replace(name,replaceWord)
    tweet=preprocessing(tweet)
    tweet=convertWord(tweet)
    X=sequence.pad_sequences([tweet],max_tweet_length)
    outcome=model.predict_classes(X)
    if(outcome==1):
        return "Not A Churn"
    else:
        return "Churn"
    return outcome



In [65]:
generatePrediction("Tired of Verizon, I hate it. I am leaving it","Verizon")

'Churn'

In [54]:
model.predict(x)

array([[0.75962645]], dtype=float32)

In [45]:
convertWord(x)

[3676]